In [1]:
%pylab
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from scipy import fftpack
import cv2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from IPython.display import IFrame, display
file = 'Lectures/FFT2.pdf'
IFrame(file, width=550, height=400)